## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
from copy import deepcopy
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

def nonmax(alpha,M):
    d1  = np.logical_or(np.logical_and(alpha<=22.5,alpha>-22.5),np.logical_and(alpha<=-157.5,alpha>157.5))
    d2 = np.logical_or(np.logical_and(alpha<=67.5,alpha>22.5),np.logical_and(alpha<=-112.5,alpha>-157.5))
    d3 = np.logical_or(np.logical_and(alpha<=112.5,alpha>67.5),np.logical_and(alpha<=-67.5,alpha>-112.5))
    d4 = np.logical_or(np.logical_and(alpha<=157.5,alpha>112.5),np.logical_and(alpha<=-22.5,alpha>-67.5))
    D = [d1,d2,d3,d4]
    gn = deepcopy(M[1:-1,1:-1]) 
    for i in range(1,len(M)-1):
        for j in range(1,len(M[0])-1):
            if max(d1[i,j],d2[i,j],d3[i,j],d4[i,j]) == d1[i,j]:
                t = d1
            if max(d1[i,j],d2[i,j],d3[i,j],d4[i,j]) == d2[i,j]:
                t = d2
            if max(d1[i,j],d2[i,j],d3[i,j],d4[i,j]) == d3[i,j]:
                t = d3
            if max(d1[i,j],d2[i,j],d3[i,j],d4[i,j]) == d4[i,j]:
                t = d4
            if np.max(t[i-1:i+2,j-1:j+2]*M[i-1:i+2,j-1:j+2]) != M[i,j]:
                gn[i-1,j-1] = 0
    return gn

def canny(img,Th,Tl,n):
    sigma = 1.7
    IG = cv2.GaussianBlur(img, (13,13), sigma)
    x_filter = np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
    y_filter = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
    G_x = np.array([[np.sum(IG[j-1:j+2,i-1:i+2]*x_filter) for i in range(1,len(img[0])-1)] for j in range(1,len(img)-1)])
    G_y = np.array([[np.sum(IG[j-1:j+2,i-1:i+2]*y_filter) for i in range(1,len(img[0])-1)] for j in range(1,len(img)-1)])
    M = np.sqrt(G_x*G_x + G_y*G_y)
    alpha = np.arctan2(G_y,G_x) * 180 / np.pi
    gn = nonmax(alpha,M)
    GH = (gn>Th)*255
    GL = np.logical_and(gn<Th,gn>Tl)*255
    stos = [[i,j] for i in range(len(GH)) for j in range(len(GH[0])) if GH[i,j]==255]
    odwiedzony = np.zeros(GH.shape)
    result = np.zeros(GH.shape)
    while stos!=[]:
        v = stos[-1]
        stos.pop()
        if not odwiedzony[v[0],v[1]]:
            odwiedzony[v[0],v[1]] = 1
            result[v[0],v[1]] = 255
            temp = [[i,j] 
            for i in range(v[0]-n if v[0]-n>=0 else v[0],v[0]+1+n if v[0]+1+n<=len(GH) else v[0]+1) 
            for j in range(v[1]-n if v[1]-n>=0 else v[1],v[1]+1+n if v[1]+1+n<=len(GH[0]) else v[1]+1) if GL[i,j]==255 and (i!=0 or j!=0)]
            stos.extend(temp)
    return result,GH


In [ ]:
if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate
I_house = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)
img,gh = canny(I_house,80,28,5)
fig,ax = plt.subplots(2,2)
fig.set_size_inches([20,20])
plt.gray()
ax[0,0].imshow(I_house)
ax[0,0].set_xticks([])
ax[0,0].set_yticks([])
ax[0,0].set_title("orginal")
ax[1,0].imshow(gh)
ax[1,0].set_xticks([])
ax[1,0].set_yticks([])
ax[1,0].set_title("GH picture")
ax[1,1].imshow(img)
ax[1,1].set_xticks([])
ax[1,1].set_yticks([])
ax[1,1].set_title("canny")
ax[0,1].imshow(cv2.Canny(I_house,190,370,None,3,1))
ax[0,1].set_xticks([])
ax[0,1].set_yticks([])
ax[0,1].set_title("cv2.canny")
plt.show()